In [89]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlalchemy

## 1 Webscrabe from Wiki information for few cities

In [91]:
def cities_dataframe(cities):
  cities_data = []

  for city in cities:
    city_data = {}

    # city
    city_data["City"] = city

    # create the soup
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # country
    city_data["Country"] = city_soup.find(class_="infobox-data").get_text()

    # population
    city_population = city_soup.find(string="Population").find_next("td").get_text()
    city_population_clean = int(city_population.replace(",", ""))
    city_data["Population"] = city_population_clean
    
    # data retrieved
    city_data["Year_Data_Retrieved"] = city_soup.find(string="Population").find_next().get_text()[2:6]

    # latitude and longitude
    city_data["Latitude"] = city_soup.find(class_="latitude").get_text()
    city_data["Longitude"] = city_soup.find(class_="longitude").get_text()

    # append this city's data to the cities list
    cities_data.append(city_data)

  return pd.DataFrame(cities_data)

In [93]:
cities_df = cities_dataframe(["Berlin", "Hamburg", "Munich"])

In [94]:
cities_df

,City,Country,Population,Year_Data_Retrieved,Latitude,Longitude
0,Berlin,Germany,3878100,2023,52°31′12″N,13°24′18″E
1,Hamburg,Germany,1964021,2023,53°33′N,10°00′E
2,Munich,Germany,1512491,2022,48°08′15″N,11°34′30″E


In [97]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   City                 3 non-null      object
 1   Country              3 non-null      object
 2   Population           3 non-null      int64 
 3   Year_Data_Retrieved  3 non-null      object
 4   Latitude             3 non-null      object
 5   Longitude            3 non-null      object
dtypes: int64(1), object(5)
memory usage: 276.0+ bytes


In [7]:
# cities_df.to_csv('cities_df.csv')

## 2 Create two tbles from the "cities_df"

Remember we want to have a relational database!!!

In [123]:
# Create the "cities df" for the relational data base

cities_to_db = cities_df[["City", "Country"]]
cities_to_db

,City,Country
0,Berlin,Germany
1,Hamburg,Germany
2,Munich,Germany


In [125]:
cities_to_db.rename(columns={"City": "City_name"}, inplace=True)
cities_to_db

C:\Users\namez\AppData\Local\Temp\ipykernel_12200\3945587001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities_to_db.rename(columns={"City": "City_name"}, inplace=True)


,City_name,Country
0,Berlin,Germany
1,Hamburg,Germany
2,Munich,Germany


In [127]:
# Create the "population df" for the relational data base

population_to_db = cities_df[["Population", "Year_Data_Retrieved"]]
population_to_db

,Population,Year_Data_Retrieved
0,3878100,2023
1,1964021,2023
2,1512491,2022


In [129]:
# cities_to_db
population_to_db

,Population,Year_Data_Retrieved
0,3878100,2023
1,1964021,2023
2,1512491,2022


## 3 Creating the database and the backbone of cities & population tables

Before we can send the information in SQL, we need to make tables that have the same columns and data types to recieve the data. While we are creating a table for cities, we can also create the population table too.

Open MySQL Workbench, open a local connection, and open a new file. Then copy and paste the code from below.

```sql
-- Drop the database if it already exists
-- DROP DATABASE IF EXISTS gans_local;


-- Create the database
CREATE DATABASE gans_local;

-- Use the database
USE gans_local;


CREATE TABLE cities (
    City_id INT AUTO_INCREMENT, -- Automatically generated ID for each city
    City_name VARCHAR(255) NOT NULL, -- Name of the city
    Country VARCHAR(255) NOT NULL, -- Name of the country
    PRIMARY KEY (City_id) -- Primary key to uniquely identify each city
);


CREATE TABLE population (
    Population_id INT AUTO_INCREMENT,
    Population INT NOT NULL,
    Year_Data_Retrieved VARCHAR(255),
    City_id INT,
    PRIMARY KEY (Population_id),
    FOREIGN KEY (City_id) REFERENCES cities(City_id)
);
```


## 4 Push the "cities_to_db" to the empty "cities" table in the database

In [10]:
# from Keys import MySQL_pass

In [115]:
cities_to_db

,City_name,Country
0,Berlin,Germany
1,Hamburg,Germany
2,Munich,Germany


In [169]:
cities_to_db

,City_name,Country
0,Berlin,Germany
1,Hamburg,Germany
2,Munich,Germany


In [171]:
# Establishment of connection with the SQL database

schema = "gans_local" # The name of your database
host = "127.0.0.1"
user = "root"
password = "PedregalNuria7178"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [173]:
# Push the "cities_to_db" to the empty "cities" table in the MySQL data base

cities_to_db.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [143]:
# Establishment of connection with the cloud SQL database

schema = "gans_cloud" # The name of your database in the cloud
host = "35.205.77.170"
user = "root"
password = "Pedregal7178cloud"
port = 3306

connection_cloud_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [176]:
cities_to_db

,City_name,Country
0,Berlin,Germany
1,Hamburg,Germany
2,Munich,Germany


In [151]:
# Push the "cities_to_db" to the empty "cities" table in the cloud MySQL data base

cities_to_db.to_sql('cities',
                  if_exists='append',
                  con=connection_cloud_string,
                  index=False)

3

## 5 Read the "cities" table from the database into the notebook

This step is needed to fetch the "city_id" column and integrate it into the "population_to_db" dataframe. The "city_id" column in "population_to_db" dataframe will serve as a foreign key in order to establish a relation between both "cities" & "population" tables

In [50]:
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql

,City_id,City_name,Country
0,1,Berlin,Germany
1,2,Hamburg,Germany
2,3,Munich,Germany
3,4,Berlin,Germany
4,5,Hamburg,Germany
5,6,Munich,Germany


In [153]:
# Getting the "City_id" to the population df 

population_to_db["City_id"] = cities_from_sql["City_id"]
population_to_db

C:\Users\namez\AppData\Local\Temp\ipykernel_12200\1642435502.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_to_db["City_id"] = cities_from_sql["City_id"]


,Population,Year_Data_Retrieved,City_id
0,3878100,2023,1
1,1964021,2023,2
2,1512491,2022,3


In [155]:
population_to_db

,Population,Year_Data_Retrieved,City_id
0,3878100,2023,1
1,1964021,2023,2
2,1512491,2022,3


In [157]:
population_to_db

,Population,Year_Data_Retrieved,City_id
0,3878100,2023,1
1,1964021,2023,2
2,1512491,2022,3


## 6 Push the "population_to_db" to the empty "population" table in the database

In [80]:
# Push the "population_to_db" to the empty "population" table in the MySQL data base

population_to_db.to_sql('population',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [159]:
# Push the "population_to_db" to the empty "cities" table in the cloud MySQL data base

population_to_db.to_sql('population',
                  if_exists='append',
                  con=connection_cloud_string,
                  index=False)

3

In [180]:
pip list

Package                           Version
--------------------------------- ------------
aext-assistant                    4.0.15
aext-assistant-server             4.0.15
aext-core                         4.0.15
aext-core-server                  4.0.15
aext-panels                       4.0.15
aext-panels-server                4.0.15
aext-share-notebook               4.0.15
aext-share-notebook-server        4.0.15
aext-shared                       4.0.15
aiobotocore                       2.7.0
aiofiles                          22.1.0
aiohttp                           3.9.3
aioitertools                      0.7.1
aiosignal                         1.2.0
aiosqlite                         0.18.0
alabaster                         0.7.12
altair                            5.0.1
anaconda-anon-usage               0.4.3
anaconda-catalogs                 0.2.0
anaconda-client                   1.12.3
anaconda-cloud-auth               0.5.1
anaconda-navigator                2.6.0
anaconda-project  